<a href="https://colab.research.google.com/github/anmaxwell/UniNotebooks/blob/master/KTrainWorking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ktrain

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import ktrain
from ktrain import text

In [ ]:
data_train = pd.read_csv('/content/train.csv')
data_train.head()
data_train.shape

(4928, 3)

In [ ]:

data_test = pd.read_csv('/content/newtest.csv')
data_test.tail()

,id,comment_text,toxic
1903,a0d0f12b99e50ba1,"""\n\nI said it because you reverted some other...",0
1904,a0d191ffa3777b3b,REDIRECT Talk:College of Basic Science and Hum...,0
1905,a0d1e66228918fcb,"Oh, I thought you were an Indian Pariah Dog fa...",0
1906,a0d23cb7e695db30,"""\n\n """"The black flag of North Vietnam""""? \n\...",0
1907,a0d2e302918727ca,its funny how Zora keeps over looking this tra...,0


In [ ]:
(X_train, y_train), (X_test, y_test), preprocess = text.texts_from_df(train_df = data_train,
                   text_column = 'comment_text',
                   label_columns=['toxic'],
                   val_df = data_test,
                   maxlen = 128,
                   preprocess_mode = 'bert')

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [ ]:
X_train[0].shape

(4928, 128)

In [ ]:
model = text.text_classifier(name = 'bert',
                             train_data = (X_train, y_train),
                             preproc=preprocess)

Is Multi-Label? False
maxlen is 128
done.


In [ ]:
learner = ktrain.get_learner(model = model,
                             train_data = (X_train, y_train),
                             val_data = (X_test, y_test),
                             batch_size = 32)

In [ ]:
learner.fit_onecycle(2e-5, 1)



begin training using onecycle policy with max lr of 2e-05...
Train on 4928 samples, validate on 1908 samples
4928/4928 [==============================] - 7360s 1s/sample - loss: 0.2280 - accuracy: 0.9123 - val_loss: 0.1149 - val_accuracy: 0.9575


In [ ]:
predictor = ktrain.get_predictor(learner.model, preprocess)

In [ ]:
data3 = ['I hate you so much right now']

In [ ]:
predictor.predict(data3)

['1']

In [ ]:
data4 = ['you are one of the most amazing people i ever met']

In [ ]:
predictor.predict(data4)

['0']

In [ ]:
data5 = ['why do you not just go to hell']

In [ ]:
predictor.predict(data5)

['1']

In [ ]:
# Saving Json file
model_json = learner.model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
# Saving Weights
learner.model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [ ]:
# Save the model for later use
predictor.save("./toxicbert")

In [ ]:
# find a good learning rate
learner.lr_find()
learner.lr_plot()


simulating training for different learning rates... this may take a few moments...
Train on 4928 samples
Epoch 1/1024
1280/4928 [======>.......................] - ETA: 1:21:47 - loss: 0.0931 - accuracy: 0.9664